<a href="https://colab.research.google.com/github/Rpmalukani22/Personalized-Medicine-Redefining-Cancer-Treatment/blob/master/Personalized_Medicine_Redefining_Cancer_Treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
#verification
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

In [5]:
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 5.3MB 25.5MB/s 
     |████████████████████████████████| 112kB 61.0MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 112kB 61.2MB/s 


In [50]:
!kaggle competitions download -c msk-redefining-cancer-treatment

training_variants.zip: Skipping, found more recently modified local copy (use --force to force download)
test_variants.zip: Skipping, found more recently modified local copy (use --force to force download)
training_text.zip: Skipping, found more recently modified local copy (use --force to force download)
test_text.zip: Skipping, found more recently modified local copy (use --force to force download)
stage2_sample_submission.csv.7z: Skipping, found more recently modified local copy (use --force to force download)
stage2_test_variants.csv.7z: Skipping, found more recently modified local copy (use --force to force download)
stage2_test_text.csv.7z: Skipping, found more recently modified local copy (use --force to force download)
stage1_solution_filtered.csv.7z: Skipping, found more recently modified local copy (use --force to force download)
stage_2_private_solution.csv.7z: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!ls -l

total 172972
-rw-r--r-- 1 root root        67 Oct 29 16:27 kaggle.json
drwxr-xr-x 1 root root      4096 Oct 25 16:58 sample_data
-rw-r--r-- 1 root root      1307 Oct 29 16:28 stage1_solution_filtered.csv.7z
-rw-r--r-- 1 root root       592 Oct 29 16:28 stage_2_private_solution.csv.7z
-rw-r--r-- 1 root root       765 Oct 29 16:28 stage2_sample_submission.csv.7z
-rw-r--r-- 1 root root   9308786 Oct 29 16:28 stage2_test_text.csv.7z
-rw-r--r-- 1 root root      7422 Oct 29 16:28 stage2_test_variants.csv.7z
-rw-r--r-- 1 root root 103787620 Oct 29 16:28 test_text.zip
-rw-r--r-- 1 root root     48614 Oct 29 16:28 test_variants.zip
-rw-r--r-- 1 root root  63917183 Oct 29 16:28 training_text.zip
-rw-r--r-- 1 root root     24831 Oct 29 16:28 training_variants.zip


In [51]:
!mkdir -p training
!unzip training_variants.zip -d ./training
!ls -l ./training

Archive:  training_variants.zip
replace ./training/training_variants? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./training/training_variants  
total 207224
-rw-r--r-- 1 root root 212125752 Jun 20  2018 training_text
-rw-rw-r-- 1 root root     66688 Jun 22  2017 training_variants


In [0]:
import pandas as pd

In [0]:
data_variants=pd.read_csv('training/training_variants')

In [54]:
data_variants.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [55]:
!unzip training_text.zip -d ./training
!ls -l ./training

Archive:  training_text.zip
replace ./training/training_text? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./training/training_text  
total 207224
-rw-r--r-- 1 root root 212125752 Jun 20  2018 training_text
-rw-rw-r-- 1 root root     66688 Jun 22  2017 training_variants


In [56]:
data_text=pd.read_csv('training/training_text',sep='''\|\|''',names=['ID','TEXT'],skiprows=1,engine='python')
data_text.head()

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [57]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [0]:
import re
def text_preprocessing(text,row,col):
  if type(text) is not int:
    text=re.sub('[^A-z0-9]',' ',str(text)) #removing special characters
    text=re.sub('\s',' ',str(text)) #replace multi character space into single character space
    text=str(text).lower()#convert into lower case
    string=""
    for i in text.split():
      if i not in stopwords:
        string+=i+" "
    data_text[col][row]=string

In [0]:
for ind,txt in data_text.iterrows():
  print(txt['TEXT'])
  if type(txt['TEXT'])is str:
    text_preprocessing(txt['TEXT'],ind,'TEXT')

In [73]:
data_text.iloc[0]['TEXT'][:100]# Print and verify

'cyclin dependent kinases cdks regulate variety fundamental cellular processes cdk10 stands one last '